In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

!pip install -q plantcv
!pip install -e .

Mounted at /content/drive
/content/drive/My Drive/Lab/Grain-U-Net
     |████████████████████████████████| 285 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 952 kB/s 
     |████████████████████████████████| 837 kB 44.8 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 140 kB 77.1 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/My%20Drive/Lab/Grain-U-Net
  Running setup.py develop for src


In [2]:
%load_ext autoreload
%autoreload 2
%aimport -tensorflow
!pip install keract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
from pathlib import Path
from skimage import io, transform
from src import get_unet, image_generator
import numpy as np

target_size = (2048,2048)             ### change
test_dir = Path('data/test_Al_AsDep_histmatched/')

path = list(test_dir.glob('*/raw/*.tif'))[0]
if path: 
    print('image found!')

img = io.imread(path)
img = transform.resize(img, target_size)
img -= img.min()
img = img.astype('float32') / np.ptp(img)
img = np.expand_dims(img, 0)


model = get_unet(input_size=(target_size + (1,)))      ### change to get_unet_dice
model.load_weights("unet_grain_nouveaux_pretune.hdf5")


image found!


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2048, 2048,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 2048, 2048,   640         ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 2048, 2048,   36928       ['conv2d[0][0]']                 
                                64)                                                           

In [ ]:
import keract

keract.get_activations(model, x=img,layer_names=['max_pooling2d', 'max_pooling2d_1', 'max_pooling2d_2', 'max_pooling2d_3', 'up_sampling2d', 'up_sampling2d_1', 'up_sampling2d_2', 'up_sampling2d_3'])

{'max_pooling2d': array([[[[2.95880944e-01, 4.68933165e-01, 0.00000000e+00, ...,
           0.00000000e+00, 2.87313282e-01, 8.23664308e-01],
          [4.20429975e-01, 4.01056558e-03, 0.00000000e+00, ...,
           0.00000000e+00, 1.01831362e-01, 0.00000000e+00],
          [3.94692838e-01, 5.81345633e-02, 0.00000000e+00, ...,
           0.00000000e+00, 1.42064214e-01, 2.02352926e-02],
          ...,
          [2.74595708e-01, 6.94614723e-02, 0.00000000e+00, ...,
           0.00000000e+00, 9.20392200e-02, 3.22906077e-02],
          [2.77009547e-01, 7.54707977e-02, 0.00000000e+00, ...,
           0.00000000e+00, 8.25919732e-02, 5.99064305e-02],
          [3.91857237e-01, 4.73395810e-02, 7.98261762e-02, ...,
           0.00000000e+00, 3.52026261e-02, 1.43827274e-02]],
 
         [[5.52911833e-02, 6.12862527e-01, 1.43045187e-03, ...,
           2.11360827e-01, 0.00000000e+00, 7.95392632e-01],
          [1.18000485e-01, 1.91543728e-01, 4.60353494e-02, ...,
           6.98162243e-02, 2.3830

<KerasTensor: shape=(None, 1024, 1024, 64) dtype=float32 (created by layer 'max_pooling2d')>